In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#os.chdir("/kaggle/input/g-research-crypto-forecasting/")
import gresearch_crypto
import traceback
from datetime import datetime
import pandas as pd
import numpy as np
#useless?
import time
import joblib
# LGBM model
from lightgbm import LGBMRegressor
# Drawing tools
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

In [ ]:
path = '/kaggle/input/g-research-crypto-forecasting/'
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "example_test.csv")
asset_details = pd.read_csv(path + "asset_details.csv")
supply_train = pd.read_csv(path + "supplemental_train.csv") # Including sample test's target

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
supply_train.head()

In [ ]:
asset_details.sort_values('Asset_ID')

In [ ]:
train.isnull().sum() # We found that the Target data has 750338 null values

In [ ]:
train = train.fillna(method='ffill')
train.isnull().sum()

## We look at the price trend of crypto with the top two assets with the highest weight

In [ ]:
btc = train[train['Asset_ID'] == 1]
btc['date'] = pd.to_datetime(btc['timestamp'],unit='s')
btc = btc.set_index("date")
btc_range = btc.iloc[-500:]
btc_range.head()

In [ ]:
ether = train[train['Asset_ID'] == 6]
ether['date'] = pd.to_datetime(ether['timestamp'],unit='s')
ether = ether.set_index("date")
ether_range = ether.iloc[-500:]
ether_range.head()

In [ ]:
# plt.figure(figsize=(16,6))
fig,axes= plt.subplots(1, 2, figsize=(16,6))
sns.heatmap(btc_range[['Count','Open','High','Low','Close','Volume','VWAP','Target']].corr(), 
            vmin=-1.0, vmax=1.0, annot=True, cmap='crest', linewidths=0.1, ax = axes[0])
sns.heatmap(ether_range[['Count','Open','High','Low','Close','Volume','VWAP','Target']].corr(), 
            vmin=-1.0, vmax=1.0, annot=True, cmap='crest', linewidths=0.1, ax = axes[1])
axes[0].set(title = "BTC Heatmap")
axes[1].set(title = "Ether Heatmap")

In [ ]:
# btc_range = btc_range.drop(['timestamp', 'Volume', 'VWAP', 'Target'], axis = 1)  # Since we cannot plot the entire btc, so selecting the latest 500 points.
# ether_range = ether_range.drop(['timestamp', 'Volume', 'VWAP', 'Target'], axis = 1)  # Since we cannot plot the entire btc, so selecting the latest 500 points.

fig = make_subplots(rows=1, cols=2, subplot_titles=["BTC", "Ether"])
fig.append_trace(go.Candlestick(x=btc_range.index, open=btc_range['Open'], high=btc_range['High'], low=btc_range['Low'], 
                                close=btc_range['Close']), row = 1, col = 1)
fig.append_trace(go.Candlestick(x=ether_range.index, open=ether_range['Open'], high=ether_range['High'], low=ether_range['Low'], 
                                close=ether_range['Close']), row = 1, col = 2)
fig.update_layout(height=400, width=1500, title_text="Price Trend")
# Although the two price ranges are different, they have similar trends

In [ ]:
fig,axes= plt.subplots(1, 2, figsize=(16,6))
sns.lineplot(data = btc['VWAP'], color = 'darkorange', label = 'BTC', ax = axes[0])
sns.lineplot(data = ether['VWAP'], color = 'darkviolet', label = 'Ether', ax = axes[1])
axes[0].set(title = "BTC Price Trend")
axes[1].set(title = "Ether Price Trend")

In [ ]:
crypto_range = train.iloc[-2800:]
crypto_range['date'] = pd.to_datetime(crypto_range['timestamp'],unit='s')
crypto_range = crypto_range.set_index("date")
fig,axes= plt.subplots(7, 2, figsize=(16,28), constrained_layout=True)
fig.suptitle('Crypto Target trend', fontsize=20)
for i in range(14):
    now_crypto = crypto_range[crypto_range.Asset_ID == asset_details.Asset_ID[i]]
    sns.lineplot(data = now_crypto, palette = 'rainbow', x = now_crypto.index, y = now_crypto.Target, label = asset_details.Asset_Name[i], ax = axes[i//2][i%2])
    axes[i//2][i%2].set(title = asset_details.Asset_Name[i] + " Price Trend")